<a href="https://colab.research.google.com/github/Datascientisit/ML_regression/blob/main/Airticket_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn

In [ ]:
train = pd.read_csv('/content/ML_train/train_data.csv (1).zip')
test = pd.read_csv('/content/ML_train/test_data.csv')
sample = pd.read_csv('/content/ML_train/sample_solution.csv')
train.head()

Ushbu maʼlumotlar toʻplami quyidagilarni o'z ichiga oladi:

Tarkib:

ID: Ketma - ketlik uchun qo'yilgan sonlar.

Airline: Parvoz qilingan aviakompaniya nomi.

Flight: Parvoz qilingan ID raqami.

Source_city: Parvoz qaysi shahardan boshlanishi.

Departure_time: Samolyotning ketish vaqti.

Stop: Parvoz davomida to'xtashlar soni.

Arrival_time: Qaytish vaqti.

Destination_city: Qaysi davlatga parvoz qilinayotgani.

Class: Foydalanuvchi samolyotning qaysi klass turidan foydalanib uchganligi.

Duration: Parvoz davomiyligi.

Days_left: Samolyotning qancha vaqtdan keyin qaytishi.

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train['class'].value_counts()

In [ ]:
train['stops'].value_counts()

In [ ]:
train['class'] = train['class'].map({'Economy': 0, 'Business': 1})
train['stops'] = train['stops'].map({'one': 1, 'zero': 0, 'two_or_more': 2})

In [ ]:
# o'rtacha qimmat va o'rtacha arzon aviakompaniyalar
train.groupby(['airline']).agg({'price': 'mean'})

In [ ]:
train['airline'] = train['airline'].map({'AirAsia': 0, 'Indigo': 1, 'GO_FIRST': 2, 'SpiceJet': 3, 'Air_India': 4, 'Vistara': 5})
train.head()

In [ ]:
train.select_dtypes(include=(int, float)).corrwith(train['price']).abs()
train.hist(bins=50, figsize=(12,8))
sns.histplot(train)

In [ ]:
print(train.isnull().sum())
print(train.shape)

Machine Learningga tayyorgarlik

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

X = train.drop('price', axis=1)
y = train['price']

scaler = StandardScaler()

nums = ['id', 'airline', 'stops', 'class', 'duration', 'days_left']
cats = ['flight', 'source_city', 'departure_time', 'arrival_time','destination_city']

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')


full_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_transformer, nums),  # Raqamli ustunlarni o'zgartirish
        ('cat', cat_transformer, cats)   # Kategorik ustunlarni o'zgartirish
    ])

X_prepared = full_pipeline.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_prepared, y, test_size=0.2, random_state=42)

In [ ]:
X_train.toarray()
X_test.toarray()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

def score(y_train, y_pred):
    mae = mean_absolute_error(y_train, y_test)
    mse = mean_squared_error(y_train, y_test)
    r2 = r2_score(y_train, y_test)
    # EVS hisoblash
    evs = explained_variance_score(y_train, y_test)
    print(f'MAE: {mae}')
    print(f'RMSE: {np.sqrt(mse)}')
    print(f'R-squared: {r2}')
    print(f"Explained Variance Score: {evs}")

Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression().fit(X_train, y_train)
y_predicted = LR_model.predict(X_test)

print('LinearRegression \'test set\'\n')
score(y_test, y_predicted)

RandomForest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor(n_estimators=50, random_state=42).fit(X_train, y_train)
y_predicted1 = RF_model.predict(X_test)

print('RF model :\n')
score(y_test, y_predicted1)

DesignTree Modeli

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DT_model = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)
y_predicted2 = DT_model.predict(X_test)

print('DT model :\n')
score(y_test, y_predicted2)

Test bilan bashorat

In [ ]:
test['class'] = test['class'].map({'Economy': 0, 'Business': 1})
test['stops'] = test['stops'].map({'one': 1, 'zero': 0, 'two_or_more': 2})

test['airline'] = test['airline'].map({'AirAsia': 0, 'Indigo': 1, 'GO_FIRST': 2, 'SpiceJet': 3, 'Air_India': 4, 'Vistara': 5})

In [ ]:
test_prepared = full_pipeline.transform(test)
test_prepared.toarray()

In [ ]:
y_predicted = RF_model.predict(test_prepared)

In [ ]:
sample = pd.read_csv('/content/ML_train/sample_solution.csv')

In [ ]:
sample['price']=y_predicted
sample.head()

In [ ]:
sample.to_csv('sample_solution.csv', index=False)